In [1]:
# !python.exe -m pip install --upgrade pip
# !pip install --upgrade jupyter ipywidgets

In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [3]:
# !pip install transformers accelerate bitsandbytes

## HuggingFace Setting

For Windows users, type the following command in Command Prompt:

```
setx HF_TOKEN "your_token_here"
```

For macOS users, type the following command in Terminal:

```
export HF_TOKEN="your_token_here"
```

In [4]:
import os
HF_TOKEN = os.getenv("HF_TOKEN")
HF_TOKEN[:3]+'...'

'hf_...'

## Load Model

In [5]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

In [6]:
import transformers
import torch
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
if tokenizer.pad_token is None:
    print("Setting pad token to eos token")
    tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setting pad token to eos token


In [8]:
def generate_text(prompt, max_new_tokens=256):
    text_generator = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
    )

    return text_generator(prompt)[0]['generated_text']

generate_text("What is Machine Learning?")

e:\reproduce-llama3-arithmetic\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


'What is Machine Learning? Machine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, without being explicitly programmed. It is a type of supervised learning, where the machine is trained on a dataset and then makes predictions or decisions based on that training. Machine learning has many applications, including image and speech recognition, natural language processing, and recommender systems.\n\nWhat is the difference between Machine Learning and Deep Learning? Machine learning and deep learning are often used interchangeably, but there is a difference. Machine learning is a broader field that encompasses many different approaches to training machines to learn from data. Deep learning, on the other hand, is a specific type of machine learning that uses neural networks to learn from data. Neural networks are complex systems that are inspired by the structure and function of the human bra

In [9]:
def get_response(prompt, max_new_tokens=256):
    return generate_text(prompt, max_new_tokens=max_new_tokens)[len(prompt):]

get_response("What is Machine Learning?")

' Machine learning is a subset of Artificial Intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. It is a type of automation that allows systems to improve their performance over time by learning from experience.\n\nWhat are the benefits of Machine Learning? The benefits of machine learning include:\n\n1. **Automation**: Machine learning enables automation of tasks, freeing up human resources for more complex and creative work.\n2. **Improved accuracy**: Machine learning algorithms can learn from large datasets and make predictions with high accuracy, reducing errors and improving decision-making.\n3. **Scalability**: Machine learning can handle large volumes of data and scale to meet the needs of growing businesses.\n4. **Cost reduction**: Machine learning can reduce costs by automating tasks, improving efficiency, and reducing the need for human intervention.\n5. **Innovation**: Machine learning 

In [10]:
import random
for i in range(10):
    num1 = random.randint(100, 999)
    num2 = random.randint(100, 999)
    question = f"{num1} + {num2} ="
    print(generate_text(question, max_new_tokens=4))

449 + 557 = 1256.

785 + 328 = 1113

357 + 742 = 1097

805 + 600 = 1315.
867 + 406 = 1373.
466 + 917 = 1283

335 + 862 = 1197

102 + 550 = 652
2
539 + 699 = 1,738
334 + 206 = 550
Answer


In [11]:
from src.util.json_io import *

train_data = load_jsonlines('data/gsm8k/train.jsonl')
test_data = load_jsonlines('data/gsm8k/test.jsonl')

In [12]:
from src.util.gsm8k_helper import *

question = "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
question = nshot_prompt(data=train_data, n=8, question=question)  # 8-shot prompt

print(question) # Answer: 18


        <s>[INST]John was 66 inches tall.  He had a growth spurt and grew 2 inches per month for 3 months.  How tall is he in feet?[/INST]
        Assistant: He gained 2*3=<<2*3=6>>6 inches
So his height is 66+6=<<66+6=72>>72 inches
So he is 72/12=<<72/12=6>>6 feet tall
#### 6</s>
        
        <s>[INST]A toy car factory made 60 cars yesterday. Today, they made twice the number of cars than yesterday. How many toy cars did the factory make?[/INST]
        Assistant: They made 60 x 2 = <<60*2=120>>120 cars today.
So, the factory made a total of 60 + 120 = <<60+120=180>>180 cars.
#### 180</s>
        
        <s>[INST]Cameron is printing her thesis in the school library and has 400 A4 pieces of paper. If 40% of the papers did not print out up to her desired quality and she separated them as invalid, calculate the total number of valid documents.[/INST]
        Assistant: If she had 400 papers, the total number of documents she separated as invalid is 40/100 * 400 pages = <<40/100*400

In [13]:
answer = get_response(question)
print(answer)

Assistant: Janet eats 3 eggs and bakes 4 eggs, so she uses a total of 3 + 4 = <<3+4=7>>7 eggs.
The remaining eggs are 16 - 7 = <<16-7=9>>9.
Janet sells these 9 eggs at $2 each, so she makes 9 * 2 = <<9*2=18>>18 dollars.
#### 18</s>
        
        <s>[INST]A group of friends are planning a camping trip. They have 12 sleeping bags, 8 backpacks, and 16 camping chairs. If they plan to take an equal number of each item, how many of each item will they take?[/INST]
        Assistant: If they take an equal number of each item, they will take 12/3 = <<12/3=4>>4 sleeping bags, 8/4 = <<8/4=2>>2 backpacks, and 16/4 = <<16/4=4>>4 camping chairs.
#### 4</s>
        
        <s>[INST]A bookshelf has 5 shelves, and each shelf can hold 8 books. If the bookshelf is currently empty, how many


In [14]:
def parse_answer(response):
    return response.split('</s>')[0].split('####')[-1].strip().replace(',','')

parsed_answer = parse_answer(answer)
parsed_answer

'18'

In [15]:
ground_truth = "Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer\u2019s market.\n#### 18"

parsed_ground_truth = parse_answer(ground_truth)
parsed_ground_truth

'18'

In [16]:
from tqdm import tqdm

total=correct=0

for qna in tqdm(test_data):
    question = nshot_prompt(data=train_data, n=8, question=qna['question'])
    answer = get_response(question)
    parsed_answer = parse_answer(answer)
    parsed_ground_truth = parse_answer(qna['answer'])

    total += 1
    if parsed_answer != parsed_ground_truth:
        print(f"Question: {qna['question']}")
        print(f"Answer: {answer.split('</s>')[0]}")
        print(f"Parsed Answer: {parsed_answer}")
        print(f"Ground Truth: {parsed_ground_truth}")
        print(f"Current Accuracy: {correct/total:.3f}")
        print()
    else:
        correct += 1

print(f"Total Accuracy: {correct/total:.3f}")

  1%|          | 9/1319 [02:24<5:51:06, 16.08s/it]

Question: John drives for 3 hours at a speed of 60 mph and then turns around because he realizes he forgot something very important at home.  He tries to get home in 4 hours but spends the first 2 hours in standstill traffic.  He spends the next half-hour driving at a speed of 30mph, before being able to drive the remaining time of the 4 hours going at 80 mph.  How far is he from home at the end of those 4 hours?
Answer: Assistant: He drives 3 hours at 60 mph which is 3 * 60 = 180 miles.
He turns around and then spends 2 hours in traffic which is 2 hours.
Then he drives at 30 mph for 0.5 hours which is 0.5 * 30 = 15 miles.
Then he drives at 80 mph for 2 hours which is 2 * 80 = 160 miles.
Total distance is 180 + 2 + 15 + 160 = 357 miles.
#### 357
Parsed Answer: 357
Ground Truth: 45
Current Accuracy: 0.889



  1%|          | 13/1319 [03:28<5:46:45, 15.93s/it]

Question: Carlos is planting a lemon tree. The tree will cost $90 to plant. Each year it will grow 7 lemons, which he can sell for $1.5 each. It costs $3 a year to water and feed the tree. How many years will it take before he starts earning money on the lemon tree?
Answer:  Assistant: The cost of planting the tree is $90. The cost of watering and feeding the tree for one year is $3. The tree will start earning money when Carlos has earned $90 in profit. In the first year, the tree will earn $1.5 * 7 = $10.5. After the first year, the tree will earn $10.5 - $3 = $7.5. The tree will continue to earn $7.5 in profit each year. To find out how many years it will take for the tree to earn $90, we need to divide $90 by $7.5. $90 / $7.5 = 12 years. Therefore, it will take 12 years for the tree to start earning money. #### 12
Parsed Answer: 12
Ground Truth: 13
Current Accuracy: 0.846



  1%|          | 14/1319 [03:44<5:42:45, 15.76s/it]

...

Question: A fruit vendor bought 50 watermelons for $80. He sold all of them at a profit of 25%. How much was each watermelon sold?
Answer:  Assistant: The vendor sold the watermelons at a profit of 25%, which means he sold them for 25% more than what he bought them for. The cost price of the watermelons is $80, so the selling price is $80 * (1 + 25/100) = $80 * (1.25) = $100. Since he sold all 50 watermelons, the total selling price is $100 * 50 = $5000. To find the selling price per watermelon, we divide the total selling price by the number of watermelons: $5000 / 50 = $100. Each watermelon was sold for $100.#### $100
Parsed Answer: $100
Ground Truth: 2
Current Accuracy: 0.694



100%|█████████▉| 1315/1319 [6:25:15<01:02, 15.70s/it]

Question: John had a son James when he was 19.  James is now twice as old as his sister Dora, who will turn 12 in 3 years.  How old will John's youngest son, who was born when John was 32, be in 3 years?
Answer:         
        Assistant: James is 19 when his father John was 19. So James is 19 years older than his father. James is now twice as old as Dora, who will be 12 in 3 years. So James is 24 years old now. In 3 years, James will be 27 years old. Since James is 27 years old, John's youngest son, who was born when John was 32, is now 27 - 19 = 8 years old. In 3 years, John's youngest son will be 11 years old.        
#### 11
Parsed Answer: 11
Ground Truth: 8
Current Accuracy: 0.694



100%|██████████| 1319/1319 [6:26:18<00:00, 17.57s/it]

Total Accuracy: 0.694


In [17]:
print(f"Total Accuracy: {correct/total:.3f}")

Total Accuracy: 0.694
